In [1]:
import os 
import json 
import re
import numpy as np

from collections import Counter, OrderedDict

np.set_printoptions(precision=2)

# Extract utilities

## Function

In [2]:
bound = 0.01
# Go from bid -> utility
def get_utility(bid, pref):
    util = 0
    issuelist = list(pref.keys())
    for issueindex, issue in enumerate(issuelist):
        # values of issue in given bid
        value = pref[issue][bid[issueindex]]
        # weight of issue in given bid
        weight = pref[issue]['weight']
        util += weight*value
    return util

# Utility change to discrete move type
def delta_mapping(delta_util):
    if (abs(delta_util[0]) <= bound and abs(delta_util[1]) <= bound):
        return 'silent'
    if (abs(delta_util[0]) <= bound and delta_util[1]) > bound:
        return 'nice'
    if (delta_util[0] <= 0 and delta_util[1] <= 0):
        return 'unfortunate'
    if (delta_util[0] > 0 and delta_util[1] <= 0):
        return 'selfish'
    if (delta_util[0] > 0 and delta_util[1] >= 0):
        return 'fortunate'
    if (delta_util[0] < 0 and delta_util[1] > 0):
        return 'concession'
    print(delta_util)

# List of bids to list of discrete moves
def discritized_mapping(agent_bids):
    mapped_utils_discrete = []
    prev_utils = agent_bids[0]
    for new_utils in agent_bids[1:]:
        delta_util1 = new_utils[0] - prev_utils[0]
        delta_util2 = new_utils[1] - prev_utils[1]
        delta_util = (delta_util1, delta_util2)
        mapped_utils_discrete.append(delta_mapping(delta_util))
        prev_utils = new_utils
    return mapped_utils_discrete
        
    
def retrieve_all_agents_bids(train):
    # Useful structures
    all_issues = train['issues']
    pref1 = train['Utility1']
    pref2 = train['Utility2']
    all_bids = train['bids']

    mapped_utils_a1 = []
    mapped_utils_a2 = []

    # Parse utility values of bids
    for bid in all_bids:
        r = bid['round']
#         print(bid)
        # stop if the negotiation session has ended
        if 'agent1' in bid:
            bid_agent1 = bid['agent1'].split(',')
            u1_b1 = get_utility(bid_agent1, pref1)
            u2_b1 = get_utility(bid_agent1, pref2)
            # Save the bid -> utility mapping
            mapped_utils_a1.append([u1_b1, u2_b1, int(r)])
        if 'agent2' in bid:
            bid_agent2 = bid['agent2'].split(',')
            u1_b2 = get_utility(bid_agent2, pref1)
            u2_b2 = get_utility(bid_agent2, pref2)
            mapped_utils_a2.append([u2_b2, u1_b2, int(r)])


    agent1_bids = [mapped_utils_a1[i][0:2] for i in range(len(mapped_utils_a1))]
    agent2_bids = [mapped_utils_a2[i][0:2] for i in range(len(mapped_utils_a2))]

    # Discritize bidspace
    agent1_bids_discrete = discritized_mapping(agent1_bids)
    agent2_bids_discrete = discritized_mapping(agent2_bids)
    
    return (agent1_bids_discrete, agent2_bids_discrete)

# Model parameters 

## Sensor model

In [3]:
path='train/'

train_files = os.listdir(path)
agent_count_mapping = {}
for t in train_files:
    train = json.load(open(os.path.join(path, t)))
    a1_name, a2_name = re.split(r'[^A-Za-z]+', t.strip('.json'))[0:2]
    a1_bids, a2_bids = retrieve_all_agents_bids(train)
    if a1_name not in agent_count_mapping:
        agent_count_mapping[a1_name] = []
    if a2_name not in agent_count_mapping:
        agent_count_mapping[a2_name] = []
    agent_count_mapping[a1_name] = agent_count_mapping[a1_name] + a1_bids
    agent_count_mapping[a2_name] = agent_count_mapping[a2_name] + a2_bids

# Start constructing the model.
sensor_model = {}
for k, v in agent_count_mapping.items():
    cnt_bids = Counter(v)
    total = len(v)
    for key in cnt_bids:
        cnt_bids[key] /= total
    sensor_model[k] = dict(cnt_bids)
    
possible_moves = ['silent', 'concession', 'unfortunate', 'nice', 'fortunate', 'selfish']

for k, moves in sensor_model.items():
    for pm in possible_moves:
        if pm not in moves:
            moves[pm] = 0.0
    sensor_model[k] = np.array(list(dict(OrderedDict(sorted(moves.items()))).values()))

evidence_index = {k: v for v, k in enumerate(sorted(possible_moves))}
state_index = {k: v for v, k in enumerate(list(sensor_model.keys()))}

sensor_model= np.array(list(sensor_model.values()))
print(state_index)
print(evidence_index)
sensor_model

{'hardheaded': 0, 'random': 1, 'conceder': 2, 'tft': 3}
{'concession': 0, 'fortunate': 1, 'nice': 2, 'selfish': 3, 'silent': 4, 'unfortunate': 5}


array([[0.04, 0.  , 0.02, 0.  , 0.9 , 0.04],
       [0.42, 0.11, 0.01, 0.34, 0.  , 0.12],
       [0.08, 0.  , 0.22, 0.  , 0.4 , 0.29],
       [0.1 , 0.01, 0.08, 0.03, 0.62, 0.16]])

## State transitions

In [6]:
state_transition = np.identity(len(state_index), dtype=float)
state_transition

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

# Filtering step

In [7]:
def predict_agent(bids):
    Bt = np.ones(4)*0.25
    for t, bt in enumerate(bids):
        sm1 = sensor_model[:, evidence_index[bt]]
        stm1 = np.dot(state_transition, Bt)
        Bt = np.multiply(sm1, stm1)
        Bt = Bt/sum(Bt)
    return Bt

In [11]:
path='train/'

train_files = os.listdir(path)
agent_count_mapping = {}
labels_keys = list(state_index.keys())

labels = []
preds = []
for t in train_files:
    train = json.load(open(os.path.join(path, t)))
    a1_name, a2_name = re.split(r'[^A-Za-z]+', t.strip('.json'))[0:2]
    a1_bids, a2_bids = retrieve_all_agents_bids(train)
    labels.append(a1_name)
    labels.append(a2_name)
    
    p1 = predict_agent(a1_bids)
    p2 = predict_agent(a2_bids)
    
    preds.append(labels_keys[np.argmax(p1)])
    preds.append(labels_keys[np.argmax(p2)])

In [20]:
c = [labels[x] == preds[x] for x in range(len(labels))]
# for i i
sum(c)/float(len(labels))

0.7647058823529411